In [13]:
#Import sql module to work with the given data
import sqlite3
import pandas as pd

In [7]:
connection = sqlite3.connect('subscriber-pipeline-starter-kit/dev/cademycode.db')

In [10]:
cursor = connection.cursor()
cursor.execute('SELECT * FROM sqlite_master').fetchone()

('table',
 'cademycode_students',
 'cademycode_students',
 2,
 'CREATE TABLE cademycode_students (\n\tuuid INTEGER, \n\tname VARCHAR, \n\tdob VARCHAR, \n\tsex TEXT, \n\tcontact_info JSON, \n\tjob_id VARCHAR, \n\tnum_course_taken VARCHAR, \n\tcurrent_career_path_id VARCHAR, \n\ttime_spent_hrs VARCHAR\n)')

#### The database contains a table with the name 'codemycode_students'

In [111]:
df = pd.read_sql_query("SELECT * FROM cademycode_students", connection)
print(df.head())

   uuid             name         dob sex  \
0     1  Annabelle Avery  1943-07-03   F   
1     2      Micah Rubio  1991-02-07   M   
2     3       Hosea Dale  1989-12-07   M   
3     4     Mariann Kirk  1988-07-31   F   
4     5  Lucio Alexander  1963-08-31   M   

                                        contact_info job_id num_course_taken  \
0  {"mailing_address": "303 N Timber Key, Irondal...    7.0              6.0   
1  {"mailing_address": "767 Crescent Fair, Shoals...    7.0              5.0   
2  {"mailing_address": "P.O. Box 41269, St. Bonav...    7.0              8.0   
3  {"mailing_address": "517 SE Wintergreen Isle, ...    6.0              7.0   
4  {"mailing_address": "18 Cinder Cliff, Doyles b...    7.0             14.0   

  current_career_path_id time_spent_hrs  
0                    1.0           4.99  
1                    8.0            4.4  
2                    8.0           6.74  
3                    9.0          12.31  
4                    3.0           5.64  


In [112]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   uuid                    5000 non-null   int64 
 1   name                    5000 non-null   object
 2   dob                     5000 non-null   object
 3   sex                     5000 non-null   object
 4   contact_info            5000 non-null   object
 5   job_id                  4995 non-null   object
 6   num_course_taken        4749 non-null   object
 7   current_career_path_id  4529 non-null   object
 8   time_spent_hrs          4529 non-null   object
dtypes: int64(1), object(8)
memory usage: 351.7+ KB
None


The columns of this database include:

uuid &emsp;&emsp;&emsp;-this just seems to be the id of the student<br>
name &emsp;&emsp;&emsp;-name of the student<br>
dob  &emsp;&emsp;&emsp;-birthday of the student<br>
sex  &emsp;&emsp;&emsp;-sex of the student<br>
contact_info &emsp;&emsp;&emsp;-dictionary object with the contact information for the student<br>
job_id &emsp;&emsp;&emsp;-integer depicting the id of the job the student holds<br>
num_course_taken &emsp;&emsp;&emsp;-number of courses the studetn is taking<br>
current_career_path_id &emsp;&emsp;&emsp;-id showing the career path the student is currently taking<br>
time_spent_hrs &emsp;&emsp;&emsp;-amount of time in hours spent on the course<br>

The current file size is 351.7+ KB. With some cleaning up, this could be reduced.

The first column that looks like it could be fixed is the sex column. This column currently is a Dtype object which takes up more space and something like a boolean. Lets first check to see how many values are in the sex column

In [113]:
print(df.sex.value_counts())

M    1995
F    1990
N    1015
Name: sex, dtype: int64


It looks like there are only three different options for sex in the database, F, M, and N. This can be simplified to a one-hot encoding object of boolean

In [114]:
sex_encoded = pd.get_dummies(df[['sex']])
df = pd.concat([df, sex_encoded], axis=1)
df = df.drop(['sex'], axis=1)
print(df.head())

   uuid             name         dob  \
0     1  Annabelle Avery  1943-07-03   
1     2      Micah Rubio  1991-02-07   
2     3       Hosea Dale  1989-12-07   
3     4     Mariann Kirk  1988-07-31   
4     5  Lucio Alexander  1963-08-31   

                                        contact_info job_id num_course_taken  \
0  {"mailing_address": "303 N Timber Key, Irondal...    7.0              6.0   
1  {"mailing_address": "767 Crescent Fair, Shoals...    7.0              5.0   
2  {"mailing_address": "P.O. Box 41269, St. Bonav...    7.0              8.0   
3  {"mailing_address": "517 SE Wintergreen Isle, ...    6.0              7.0   
4  {"mailing_address": "18 Cinder Cliff, Doyles b...    7.0             14.0   

  current_career_path_id time_spent_hrs  sex_F  sex_M  sex_N  
0                    1.0           4.99      1      0      0  
1                    8.0            4.4      0      1      0  
2                    8.0           6.74      0      1      0  
3                    9.0  

Now the sex column has been converted into three columns that should be a little more efficient to work with. Lets check

In [115]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   uuid                    5000 non-null   int64 
 1   name                    5000 non-null   object
 2   dob                     5000 non-null   object
 3   contact_info            5000 non-null   object
 4   job_id                  4995 non-null   object
 5   num_course_taken        4749 non-null   object
 6   current_career_path_id  4529 non-null   object
 7   time_spent_hrs          4529 non-null   object
 8   sex_F                   5000 non-null   uint8 
 9   sex_M                   5000 non-null   uint8 
 10  sex_N                   5000 non-null   uint8 
dtypes: int64(1), object(7), uint8(3)
memory usage: 327.3+ KB
None


Now we are down to 327.3 KB
Next I would like to take a look at the different job_ids. It will be interesting to see what the different jobs are in this dataset.

In [116]:
print(df.job_id.value_counts())

2.0    706
1.0    693
7.0    680
3.0    675
4.0    671
5.0    660
6.0    657
8.0    253
Name: job_id, dtype: int64


The job_id is only 8 different values in the type object. Before we convert this to an integer instead of an object, there are 5 rows that have Null values for this. I would like to convert these values to a 0 to make the transition easier. But first I would like to take a look at the 5 rows just to see if I notice anything odd about them.

In [117]:
null_mask = df['job_id'].isnull()
null_rows = df[null_mask]
print(null_rows)

      uuid              name         dob  \
162    163        Glen Riley  2002-08-22   
757    758  Mercedez Vorberg  2002-03-25   
854    855           Kurt Ho  2002-05-29   
1029  1030      Penny Gaines  2002-03-01   
1542  1543  Frederick Reilly  2002-11-13   

                                           contact_info job_id  \
162   {"mailing_address": "P.O. Box 37267, Cornlea v...   None   
757   {"mailing_address": "284 Cedar Seventh, Virden...   None   
854   {"mailing_address": "P.O. Box 27254, Olin, New...   None   
1029  {"mailing_address": "138 Misty Vale, Stockton ...   None   
1542  {"mailing_address": "P.O. Box 40769, Quakervil...   None   

     num_course_taken current_career_path_id time_spent_hrs  sex_F  sex_M  \
162               8.0                    3.0            5.7      0      1   
757              15.0                    4.0           4.14      1      0   
854               0.0                    8.0          23.72      0      1   
1029             15.0         

There doesnt seem to be anything special about the missing values. I am going to change the values from None to 0 and make a note in the readme

In [118]:
df['job_id'] = df['job_id'].fillna(0.0)
df['job_id'] = df['job_id'].astype('float')
df['job_id'] = df['job_id'].astype('int')
print(df.job_id.value_counts())
print(df.info())

2    706
1    693
7    680
3    675
4    671
5    660
6    657
8    253
0      5
Name: job_id, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   uuid                    5000 non-null   int64 
 1   name                    5000 non-null   object
 2   dob                     5000 non-null   object
 3   contact_info            5000 non-null   object
 4   job_id                  5000 non-null   int32 
 5   num_course_taken        4749 non-null   object
 6   current_career_path_id  4529 non-null   object
 7   time_spent_hrs          4529 non-null   object
 8   sex_F                   5000 non-null   uint8 
 9   sex_M                   5000 non-null   uint8 
 10  sex_N                   5000 non-null   uint8 
dtypes: int32(1), int64(1), object(6), uint8(3)
memory usage: 307.7+ KB
None


Now we have reduced the memory allocated to job_id to display the values as integers. This reduced our memory used from 327KB to 307KB.

Next, to take care of the other low hanging fruit, I can change the data type for name from an object to a text and change the dob from an object to a date datatype

In [119]:
df['name'] = df['name'].astype('str')
df['dob'] = pd.to_datetime(df['dob'], format='%Y-%m-%d')

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    5000 non-null   int64         
 1   name                    5000 non-null   object        
 2   dob                     5000 non-null   datetime64[ns]
 3   contact_info            5000 non-null   object        
 4   job_id                  5000 non-null   int32         
 5   num_course_taken        4749 non-null   object        
 6   current_career_path_id  4529 non-null   object        
 7   time_spent_hrs          4529 non-null   object        
 8   sex_F                   5000 non-null   uint8         
 9   sex_M                   5000 non-null   uint8         
 10  sex_N                   5000 non-null   uint8         
dtypes: datetime64[ns](1), int32(1), int64(1), object(5), uint8(3)
memory usage: 307.7+ KB
None


It looks like this didnt really reduce the memory of the dataset. However, the Dtypes are a little more descriptive now.

The next column I will look at is the num_courses_taken column. First, lets see what types of values are stored in this column

In [120]:
print(df.num_course_taken.value_counts())

5.0     341
12.0    332
2.0     312
15.0    309
10.0    306
7.0     303
13.0    297
0.0     296
8.0     291
11.0    289
4.0     285
6.0     282
14.0    280
3.0     279
1.0     279
9.0     268
Name: num_course_taken, dtype: int64


It looks like this is another column where we could convert to integers. Lets take a look at the rows that have missing values

In [121]:
null_mask = df['num_course_taken'].isnull()
null_rows = df[null_mask]
print(null_rows)

      uuid             name        dob  \
25      26    Doug Browning 1970-06-08   
26      27  Damon Schrauwen 1953-10-31   
51      52       Alisa Neil 1977-05-28   
70      71  Chauncey Hooper 1962-04-07   
80      81  Ellyn van Heest 1984-06-28   
...    ...              ...        ...   
4889  4890    Tegan Cochran 1970-11-08   
4898  4899   Ruthann Oliver 1998-05-22   
4914  4915    Ernest Holmes 1995-03-11   
4980  4981   Brice Franklin 1946-12-01   
4985  4986     Russel Vonck 1994-09-07   

                                           contact_info  job_id  \
25    {"mailing_address": "P.O. Box 15845, Devine, F...       7   
26    {"mailing_address": "P.O. Box 84659, Maben, Ge...       4   
51    {"mailing_address": "16 View Annex, Mosses, No...       5   
70    {"mailing_address": "955 Dewy Flat, Slaughterv...       3   
80    {"mailing_address": "872 Cider Glade, Chicken,...       3   
...                                                 ...     ...   
4889  {"mailing_address": 

There doesnt seem to be any reason why this data is mising. The other columns are populated. So for now, I will convert these to np.nan so I can convert the rest of the column to integers

In [122]:
df['num_course_taken'] = df['num_course_taken'].astype('float')
df['num_course_taken'] = df['num_course_taken'].astype('Int32')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    5000 non-null   int64         
 1   name                    5000 non-null   object        
 2   dob                     5000 non-null   datetime64[ns]
 3   contact_info            5000 non-null   object        
 4   job_id                  5000 non-null   int32         
 5   num_course_taken        4749 non-null   Int32         
 6   current_career_path_id  4529 non-null   object        
 7   time_spent_hrs          4529 non-null   object        
 8   sex_F                   5000 non-null   uint8         
 9   sex_M                   5000 non-null   uint8         
 10  sex_N                   5000 non-null   uint8         
dtypes: Int32(1), datetime64[ns](1), int32(1), int64(1), object(4), uint8(3)
memory usage: 293.1+ KB
None


Now the num_course_taken column will have null values but wont be an object which will make it easier for us to work with. This also reduced our memory usage from 307KB to 293KB! Now I would like to try the same thing with the current_career_path_id column

In [123]:
print(df.current_career_path_id.value_counts())

5.0     476
3.0     469
10.0    460
1.0     459
6.0     454
2.0     450
7.0     449
9.0     441
8.0     437
4.0     434
Name: current_career_path_id, dtype: int64


In [124]:
df['current_career_path_id'] = df['current_career_path_id'].astype('float')
df['current_career_path_id'] = df['current_career_path_id'].astype('Int32')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    5000 non-null   int64         
 1   name                    5000 non-null   object        
 2   dob                     5000 non-null   datetime64[ns]
 3   contact_info            5000 non-null   object        
 4   job_id                  5000 non-null   int32         
 5   num_course_taken        4749 non-null   Int32         
 6   current_career_path_id  4529 non-null   Int32         
 7   time_spent_hrs          4529 non-null   object        
 8   sex_F                   5000 non-null   uint8         
 9   sex_M                   5000 non-null   uint8         
 10  sex_N                   5000 non-null   uint8         
dtypes: Int32(2), datetime64[ns](1), int32(1), int64(1), object(3), uint8(3)
memory usage: 278.4+ KB
None


That took our memory usage down from 293 to 278! Keep in mind we started at 350KB.
Lastly, we will try this out wiht the time_spend_hrs column

In [125]:
print(df.time_spent_hrs.value_counts())

5.93     8
17.47    8
11.9     7
7.05     7
2.91     7
        ..
27.53    1
8.07     1
27.51    1
29.66    1
23.54    1
Name: time_spent_hrs, Length: 2192, dtype: int64


It looks like time spent in hours has 2192 unique values, and there are quite a few with decimal value which is important. Lets just convert these to a float value and leave it as is.

In [126]:
df['time_spent_hrs'] = df['time_spent_hrs'].astype('float32')
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   uuid                    5000 non-null   int64         
 1   name                    5000 non-null   object        
 2   dob                     5000 non-null   datetime64[ns]
 3   contact_info            5000 non-null   object        
 4   job_id                  5000 non-null   int32         
 5   num_course_taken        4749 non-null   Int32         
 6   current_career_path_id  4529 non-null   Int32         
 7   time_spent_hrs          4529 non-null   float32       
 8   sex_F                   5000 non-null   uint8         
 9   sex_M                   5000 non-null   uint8         
 10  sex_N                   5000 non-null   uint8         
dtypes: Int32(2), datetime64[ns](1), float32(1), int32(1), int64(1), object(2), uint8(3)
memory usage: 258.9+ KB


In [129]:
df.time_spent_hrs.value_counts()

5.930000     8
17.469999    8
11.900000    7
7.050000     7
2.910000     7
            ..
27.530001    1
8.070000     1
27.510000    1
29.660000    1
23.540001    1
Name: time_spent_hrs, Length: 2192, dtype: int64

Now the time_spend_hrs is type float32 which is the lowest we can go without losing precision of our data. This is the cleanest we can get with the data for now

In [130]:
df.describe()

,uuid,job_id,num_course_taken,current_career_path_id,time_spent_hrs,sex_F,sex_M,sex_N
count,5000.000000,5000.00000,4749.0,4529.0,4529.000000,5000.000000,5000.000000,5000.000000
mean,2500.500000,4.16800,7.533586,5.478914,11.520624,0.398000,0.399000,0.203000
std,1443.520003,2.15107,4.609884,2.874135,7.564236,0.489534,0.489742,0.402273
min,1.000000,0.00000,0.0,1.0,0.000000,0.000000,0.000000,0.000000
25%,1250.750000,2.00000,4.0,3.0,5.380000,0.000000,0.000000,0.000000
50%,2500.500000,4.00000,7.0,5.0,10.670000,0.000000,0.000000,0.000000
75%,3750.250000,6.00000,12.0,8.0,16.750000,1.000000,1.000000,0.000000
max,5000.000000,8.00000,15.0,10.0,35.980000,1.000000,1.000000,1.000000


In [135]:
print(df[df['time_spent_hrs'].isna()])

      uuid                   name        dob  \
15      16          Norene Dalton 1976-04-30   
19      20  Sofia van Steenbergen 1990-02-21   
30      31      Christoper Warner 1989-12-28   
49      50           Antony Horne 1996-05-29   
54      55              Omar Bunk 1955-11-08   
...    ...                    ...        ...   
4904  4905         Eduardo Daniel 2004-06-18   
4922  4923      Francisco van Ede 1961-04-26   
4948  4949      Dewitt van Malsem 1949-03-08   
4956  4957          Todd Stamhuis 1961-06-15   
4974  4975         Jorge Creelman 1944-11-24   

                                           contact_info  job_id  \
15    {"mailing_address": "130 Wishing Essex, Branch...       6   
19    {"mailing_address": "634 Clear Barn Dell, Beam...       7   
30    {"mailing_address": "556 Stony Highlands, Drai...       2   
49    {"mailing_address": "P.O. Box 78685, Lenox, Te...       3   
54    {"mailing_address": "445 Dale Hollow, Vermont ...       3   
...                  

Upon looking deeper into the data, it looks like there are quite a few rows where both current_career_path_id and time_spend_hrs are both missing. Both rows have the same number of non null values, but I'm not sure if all rows with time_spend_hrs as NaN is the same rows as the current_career_path_id as NA

In [139]:
time_spend_na = df[df['time_spent_hrs'].isna()]
time_spend_na.describe()

,uuid,job_id,num_course_taken,current_career_path_id,time_spent_hrs,sex_F,sex_M,sex_N
count,471.000000,471.000000,451.0,0.0,0.0,471.000000,471.000000,471.000000
mean,2550.690021,4.405520,7.776053,<NA>,NaN,0.397028,0.435244,0.167728
std,1412.471144,2.093326,4.605403,<NA>,NaN,0.489802,0.496316,0.374022
min,16.000000,1.000000,0.0,<NA>,NaN,0.000000,0.000000,0.000000
25%,1370.500000,3.000000,4.0,<NA>,NaN,0.000000,0.000000,0.000000
50%,2494.000000,5.000000,8.0,<NA>,NaN,0.000000,0.000000,0.000000
75%,3802.000000,6.000000,12.0,<NA>,NaN,1.000000,1.000000,0.000000
max,4975.000000,8.000000,15.0,<NA>,NaN,1.000000,1.000000,1.000000


In [140]:
career_path_na = df[df['current_career_path_id'].isna()]
career_path_na.describe()

,uuid,job_id,num_course_taken,current_career_path_id,time_spent_hrs,sex_F,sex_M,sex_N
count,471.000000,471.000000,451.0,0.0,0.0,471.000000,471.000000,471.000000
mean,2550.690021,4.405520,7.776053,<NA>,NaN,0.397028,0.435244,0.167728
std,1412.471144,2.093326,4.605403,<NA>,NaN,0.489802,0.496316,0.374022
min,16.000000,1.000000,0.0,<NA>,NaN,0.000000,0.000000,0.000000
25%,1370.500000,3.000000,4.0,<NA>,NaN,0.000000,0.000000,0.000000
50%,2494.000000,5.000000,8.0,<NA>,NaN,0.000000,0.000000,0.000000
75%,3802.000000,6.000000,12.0,<NA>,NaN,1.000000,1.000000,0.000000
max,4975.000000,8.000000,15.0,<NA>,NaN,1.000000,1.000000,1.000000


It seems like the missing data in the current_career_path_id and the time_spend_hrs are related. This could potentially be for new users who maybe signed up for a course but havent started yet.